In [1]:
from pypdf import PdfReader

In [2]:
import os

In [189]:
from dotenv import load_dotenv

In [191]:
load_dotenv()

True

In [193]:
from openai import OpenAI

In [194]:
import os

In [195]:
import chromadb

In [196]:
chroma_client = chromadb.PersistentClient(path = "/Users/Rakshit.Lodha/doc-q-a")

In [237]:
collection_1 = chroma_client.create_collection(name = "doc-qa-1")

In [239]:
collection_1.count()

0

In [200]:
client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))

In [201]:
import pandas as pd

In [202]:
import numpy as np

In [203]:
from tqdm import tqdm

In [4]:
pdf_file_path = "/Users/Rakshit.Lodha/doc-q-a/scan.pdf"

In [22]:
with open(pdf_file_path, 'rb') as pdf_file:
    reader = PdfReader("scan.pdf")
    number_of_pages = len(reader.pages)

In [44]:
text = " "

for page in reader.pages:
    text += page.extract_text() + '\n'

In [56]:
lines = text.split("\n")

lines

[' A Scandal in Bohemia',
 'Arthur Conan Doyle',
 'This text is provided to you “as-is” without any warranty. No warranties of any kind, expressed or implied, are made to you as to the',
 'text or any medium it may be on, including but not limited to warranties of merchantablity or ﬁtness for a particular purpose.',
 'This text was formatted from various free ASCII and HTML variants. See http:/ /sherlock-holm.es for an electronic form of this text',
 'and additional information about it.',
 'This text comes from the collection’s version 3.1.',
 'Table of contents',
 'Chapter 1. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .3',
 'Chapter 2. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .7',
 'Chapter 3. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .12',
 '1',
 '',
 'CHAPTE

In [80]:
processed_lines = []

i = 0

while i < len(lines):
    current_line = lines[i].strip()

    if current_line.endswith('-'):

        next_line = lines[i+1].strip()
        merged_line = current_line[:-1] + next_line

        processed_lines.append(merged_line)

        i +=2
    else:
        processed_lines.append(lines[i].strip())
        i +=1

processed_lines

['A Scandal in Bohemia',
 'Arthur Conan Doyle',
 'This text is provided to you “as-is” without any warranty. No warranties of any kind, expressed or implied, are made to you as to the',
 'text or any medium it may be on, including but not limited to warranties of merchantablity or ﬁtness for a particular purpose.',
 'This text was formatted from various free ASCII and HTML variants. See http:/ /sherlock-holm.es for an electronic form of this text',
 'and additional information about it.',
 'This text comes from the collection’s version 3.1.',
 'Table of contents',
 'Chapter 1. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .3',
 'Chapter 2. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .7',
 'Chapter 3. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .12',
 '1',
 '',
 'CHAPTER

In [134]:
paragraphs = []

current_paragraph = ""

for line in processed_lines:
    if line.endswith(('?','.', '!')):
        current_paragraph += line
        paragraphs.append(current_paragraph)
        current_paragraph = ""
    else:
        current_paragraph += line + " "

final_paragraphs = paragraphs[4:]

In [135]:
final_paragraphs

['T o Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position. He never spoke of the softer passions, save with a gibe and a sneer. They were admirable things for the observer—excellent for drawing the veil from men’s motives and actions. But for the trained reasoner to admit such intrusions into his own delicate and ﬁnely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results. Grit in a sensitive instrument, or a crack in one of his own high-power lenses, would not be more disturbing than a strong em

In [185]:
joined = []

char = ""

for para in final_paragraphs:
    if len(char) + len(para) <= 1000:
        char += para + " "
    elif len(char) + len(para)>1000:
        joined.append(char)
        char = para

if char:
    joined.append(char)

In [236]:
final_output = joined[1:]

In [184]:
len(joined)

54

In [211]:
def create_embeddings(text, batch_size = 50):
    all_embeddings = []

    for i in tqdm(range(0, len(text), batch_size), desc = "Creating Embeddings"):
        batch = text[i:i+batch_size]

        embeddings = client.embeddings.create(
            model = "text-embedding-3-small",
            input = batch
        )

        batch_embeddings = [item.embedding for item in embeddings.data]

        all_embeddings.extend(batch_embeddings)

    return all_embeddings

In [213]:
final_emb = create_embeddings(final_output)

Creating Embeddings: 100%|████████████████████████| 2/2 [00:03<00:00,  1.85s/it]


In [215]:
def id_creation(text):

    final_id = []

    for index, element in enumerate(text, start = 1):
        ids = f"id_{index}"
        final_id.append(ids)

    return final_id

In [216]:
id_created = id_creation(final_output)

In [240]:
def chroma_db_creation():

    batch_size = 50

    for i in tqdm(range(0, len(id_created), batch_size), desc = "Adding to Chroma"):
        batch_ids = id_created[i:i+batch_size]
        batch_embeddings = final_emb[i:i+batch_size]
        batch_documents = final_output[i:i+batch_size]

        collection_1.add(
            ids = batch_ids,
            embeddings = batch_embeddings,
            documents=batch_documents
        )

In [241]:
chroma_db_creation()

Adding to Chroma: 100%|███████████████████████████| 2/2 [00:00<00:00, 18.30it/s]


In [242]:
def query(text, results=3):

    query_conversion = client.embeddings.create(
        model = "text-embedding-3-small",
        input = text
    )

    result = collection_1.query(
        query_embeddings = query_conversion.data[0].embedding,
        n_results = results
    )

    return result
    

In [245]:
query("Who is Irene?",5)

{'ids': [['id_1', 'id_18', 'id_26', 'id_53', 'id_39']],
 'embeddings': None,
 'documents': [['T o Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position. He never spoke of the softer passions, save with a gibe and a sneer. They were admirable things for the observer—excellent for drawing the veil from men’s motives and actions. But for the trained reasoner to admit such intrusions into his own delicate and ﬁnely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results. Grit in a sensitive instrument, o